<a href="https://colab.research.google.com/github/shitizz24/Time-Series-Forecasting/blob/master/Support_Vector_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
# import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
from numpy.random import seed
seed(1)

In [6]:
dataset=pd.read_csv("Data/testset.csv")
dataset

,datetime_utc,_conds,_dewptm,_fog,_hail,_heatindexm,_hum,_precipm,_pressurem,_rain,_snow,_tempm,_thunder,_tornado,_vism,_wdird,_wdire,_wgustm,_windchillm,_wspdm
0,19961101-11:00,Smoke,9.0,0,0,NaN,27.0,NaN,1010.0,0,0,30.0,0,0,5.00,280.0,West,NaN,NaN,7.4
1,19961101-12:00,Smoke,10.0,0,0,NaN,32.0,NaN,-9999.0,0,0,28.0,0,0,NaN,0.0,North,NaN,NaN,NaN
2,19961101-13:00,Smoke,11.0,0,0,NaN,44.0,NaN,-9999.0,0,0,24.0,0,0,NaN,0.0,North,NaN,NaN,NaN
3,19961101-14:00,Smoke,10.0,0,0,NaN,41.0,NaN,1010.0,0,0,24.0,0,0,2.00,0.0,North,NaN,NaN,NaN
4,19961101-16:00,Smoke,11.0,0,0,NaN,47.0,NaN,1011.0,0,0,23.0,0,0,1.20,0.0,North,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99565,20161231-12:00,Partly Cloudy,15.0,0,0,NaN,77.0,NaN,1015.0,0,0,18.0,0,0,0.50,110.0,ESE,NaN,NaN,7.4
99566,20161231-13:00,Partly Cloudy,16.0,0,0,NaN,100.0,NaN,1014.0,0,0,16.0,0,0,0.80,90.0,East,NaN,NaN,3.7
99567,20161231-15:00,NaN,13.0,0,0,NaN,NaN,NaN,1016.0,0,0,NaN,0,0,0.05,10.0,North,NaN,NaN,59.3
99568,20161231-18:00,Patches of Fog,12.0,1,0,NaN,100.0,NaN,1016.0,0,0,12.0,0,0,0.50,NaN,NaN,NaN,NaN,0.0


In [7]:
dataset.columns=dataset.columns.str.replace("_","")
dataset.columns=dataset.columns.str.replace(" ","")
dataset.columns

Index(['datetimeutc', 'conds', 'dewptm', 'fog', 'hail', 'heatindexm', 'hum',
       'precipm', 'pressurem', 'rain', 'snow', 'tempm', 'thunder', 'tornado',
       'vism', 'wdird', 'wdire', 'wgustm', 'windchillm', 'wspdm'],
      dtype='object')

In [8]:
dataset.drop(columns=["conds","dewptm","fog","hail","heatindexm","precipm","rain","snow","thunder","tornado","vism","wdird","wdire","wgustm","windchillm"],inplace=True,axis=1)
dataset

,datetimeutc,hum,pressurem,tempm,wspdm
0,19961101-11:00,27.0,1010.0,30.0,7.4
1,19961101-12:00,32.0,-9999.0,28.0,NaN
2,19961101-13:00,44.0,-9999.0,24.0,NaN
3,19961101-14:00,41.0,1010.0,24.0,NaN
4,19961101-16:00,47.0,1011.0,23.0,0.0
...,...,...,...,...,...
99565,20161231-12:00,77.0,1015.0,18.0,7.4
99566,20161231-13:00,100.0,1014.0,16.0,3.7
99567,20161231-15:00,NaN,1016.0,NaN,59.3
99568,20161231-18:00,100.0,1016.0,12.0,0.0


In [9]:
dataset['pressurem'].replace('-9999.0',np.nan)
dataset.replace(0,np.nan)
dataset.isnull().sum()

datetimeutc       0
hum             756
pressurem       232
tempm           672
wspdm          2358
dtype: int64

In [10]:
dataset

,datetimeutc,hum,pressurem,tempm,wspdm
0,19961101-11:00,27.0,1010.0,30.0,7.4
1,19961101-12:00,32.0,-9999.0,28.0,NaN
2,19961101-13:00,44.0,-9999.0,24.0,NaN
3,19961101-14:00,41.0,1010.0,24.0,NaN
4,19961101-16:00,47.0,1011.0,23.0,0.0
...,...,...,...,...,...
99565,20161231-12:00,77.0,1015.0,18.0,7.4
99566,20161231-13:00,100.0,1014.0,16.0,3.7
99567,20161231-15:00,NaN,1016.0,NaN,59.3
99568,20161231-18:00,100.0,1016.0,12.0,0.0


In [11]:
hum_median=dataset["hum"].median()
dataset["hum"].fillna(hum_median,inplace=True)
pressurem_median=dataset["pressurem"].median()
dataset["pressurem"].fillna(pressurem_median,inplace=True)
wspdm_median=dataset["wspdm"].median()
dataset["wspdm"].fillna(wspdm_median,inplace=True)
tempm_median=dataset["tempm"].median()
dataset["tempm"].fillna(tempm_median,inplace=True)

In [12]:
dataset.isnull().sum()

datetimeutc    0
hum            0
pressurem      0
tempm          0
wspdm          0
dtype: int64

In [13]:
dataset

,datetimeutc,hum,pressurem,tempm,wspdm
0,19961101-11:00,27.0,1010.0,30.0,7.4
1,19961101-12:00,32.0,-9999.0,28.0,7.4
2,19961101-13:00,44.0,-9999.0,24.0,7.4
3,19961101-14:00,41.0,1010.0,24.0,7.4
4,19961101-16:00,47.0,1011.0,23.0,0.0
...,...,...,...,...,...
99565,20161231-12:00,77.0,1015.0,18.0,7.4
99566,20161231-13:00,100.0,1014.0,16.0,3.7
99567,20161231-15:00,58.0,1016.0,27.0,59.3
99568,20161231-18:00,100.0,1016.0,12.0,0.0


In [14]:
def extract_year(value):
    return (value[0:4])

In [15]:
def extract_month(value):
    return (value[4:6])

In [16]:
def extract_date(value):
    return (value[6:8])

In [17]:
dataset["year"]=dataset["datetimeutc"].apply(lambda x : extract_year(x))

In [18]:
dataset["month"]=dataset["datetimeutc"].apply(lambda x : extract_month(x))

In [19]:
dataset["date"]=dataset["datetimeutc"].apply(lambda x : extract_month(x))

In [20]:
dataset.year.value_counts()

2003    13186
2004     9151
2002     8503
2001     8403
2016     6634
1997     6513
1999     6300
2000     5391
1998     3579
2014     2973
2015     2936
2005     2920
2007     2865
2006     2858
2008     2857
2009     2851
2010     2768
2011     2738
2012     2438
2013     2428
1996     1278
Name: year, dtype: int64

In [21]:
dataset.groupby("year").tempm.mean()

year
1996    16.658059
1997    24.477199
1998    24.872031
1999    27.333333
2000    23.884252
2001    25.599667
2002    26.487005
2003    26.182239
2004    26.010054
2005    24.720548
2006    25.455213
2007    24.813264
2008    24.527476
2009    25.358821
2010    25.559610
2011    24.968590
2012    25.028712
2013    24.690692
2014    25.184326
2015    25.148161
2016    27.314591
Name: tempm, dtype: float64

In [22]:
dataset

,datetimeutc,hum,pressurem,tempm,wspdm,year,month,date
0,19961101-11:00,27.0,1010.0,30.0,7.4,1996,11,11
1,19961101-12:00,32.0,-9999.0,28.0,7.4,1996,11,11
2,19961101-13:00,44.0,-9999.0,24.0,7.4,1996,11,11
3,19961101-14:00,41.0,1010.0,24.0,7.4,1996,11,11
4,19961101-16:00,47.0,1011.0,23.0,0.0,1996,11,11
...,...,...,...,...,...,...,...,...
99565,20161231-12:00,77.0,1015.0,18.0,7.4,2016,12,12
99566,20161231-13:00,100.0,1014.0,16.0,3.7,2016,12,12
99567,20161231-15:00,58.0,1016.0,27.0,59.3,2016,12,12
99568,20161231-18:00,100.0,1016.0,12.0,0.0,2016,12,12


In [23]:
dataset_svr=dataset[['hum','tempm','pressurem','wspdm','year','month','date']]

In [24]:
dataset_svr

,hum,tempm,pressurem,wspdm,year,month,date
0,27.0,30.0,1010.0,7.4,1996,11,11
1,32.0,28.0,-9999.0,7.4,1996,11,11
2,44.0,24.0,-9999.0,7.4,1996,11,11
3,41.0,24.0,1010.0,7.4,1996,11,11
4,47.0,23.0,1011.0,0.0,1996,11,11
...,...,...,...,...,...,...,...
99565,77.0,18.0,1015.0,7.4,2016,12,12
99566,100.0,16.0,1014.0,3.7,2016,12,12
99567,58.0,27.0,1016.0,59.3,2016,12,12
99568,100.0,12.0,1016.0,0.0,2016,12,12


In [25]:
y=dataset_svr["tempm"]
y

0        30.0
1        28.0
2        24.0
3        24.0
4        23.0
         ... 
99565    18.0
99566    16.0
99567    27.0
99568    12.0
99569    12.0
Name: tempm, Length: 99570, dtype: float64

In [26]:
dataset_svr.drop("tempm",inplace=True,axis=1)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [27]:
dataset_svr.columns

Index(['hum', 'pressurem', 'wspdm', 'year', 'month', 'date'], dtype='object')

In [28]:
X=dataset_svr.iloc[:,:].values
X.shape

(99570, 6)

In [29]:
y=y.values
y.shape

(99570,)

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.10,random_state=0)

In [32]:
print("Shape of X_train", X_train.shape)
print("Shape of X_test", X_test.shape)
print("Shape of y_train", y_train.shape)
print("Shape of y_test", y_test.shape)

Shape of X_train (89613, 6)
Shape of X_test (9957, 6)
Shape of y_train (89613,)
Shape of y_test (9957,)


In [33]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')

In [34]:
regressor.fit(X_train,y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [36]:
y_pred=regressor.predict(X_test)

In [37]:
y_pred

array([27.10015282, 27.10030338, 27.0997921 , ..., 27.0861523 ,
       27.09933174, 27.0996197 ])

In [38]:
y_test

array([32., 34., 16., ..., 32., 12., 25.])

In [39]:
# Accuracy
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse=mean_squared_error(y_test,y_pred)
mae=mean_absolute_error(y_test,y_pred)
print(f'{mse} {mae}')

73.70098640072135 6.830607095328833
